In [1]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-11-14 15:25:40.182294: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 15:25:40.190564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 15:25:40.199658: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 15:25:40.202465: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 15:25:40.209554: I tensorflow/core/platform/cpu_feature_guar

# Transformation de toutes les images en JPEG

In [3]:
import pathlib

from pathlib import PosixPath

from PIL import Image

data_dir_path = '/home/shared/Mammiferes'


def transform_image_type_to_jpg(images: list[PosixPath]) -> None:
    i = 0
    for image in images:
        img = Image.open(image)
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
        # The image object is used to save the image in jpg format
        img.save(image, 'JPEG')
        i += 1


data_dir = pathlib.Path(data_dir_path).with_suffix('')

image_paths = list(data_dir.glob('*/*.*'))

transform_image_type_to_jpg(image_paths)

image_count = len(image_paths)

print(image_count)


252


# Data Augmentation

In [11]:
from matplotlib import pyplot as plt
from tensorflow.python.data import AUTOTUNE
from tensorflow.data import Dataset
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import pandas as pd
import tensorflow_datasets as tfds

batch_size = 32


def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1, 2, 1)
    plt.title('Original image')
    plt.imshow(original)

    plt.subplot(1, 2, 2)
    plt.title('Augmented image')
    plt.imshow(augmented)


(train_datasets, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

image_width, image_height = (256, 256)

rng = tf.random.Generator.from_seed(123, alg='philox')


def resize_and_rescale(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [256, 256])
    image = (image / 255.0)
    return image, label


def augment_and_make_seed(x, y):
    print(x)
    print(y)
    seed = rng.make_seeds(1)[:, 0]
    image, label = augment((x, y), seed)
    return image, label


def augment(image_label, seed):
    image, label = image_label
    image, label = resize_and_rescale(image, label)
    image = tf.image.resize_with_crop_or_pad(image, image_width + 6, image_height + 6)
    # Make a new seed.
    new_seed = tf.random.split(seed, num=1)[0, :]
    # Random crop back to the original size.
    image = tf.image.stateless_random_crop(
        image, size=[image_width, image_height, 3], seed=seed)
    # Random brightness.
    image = tf.image.stateless_random_brightness(
        image, max_delta=0.5, seed=new_seed)
    image = tf.clip_by_value(image, 0, 1)
    return image, label


def apply_augmentation(dataset: Dataset):
    mapped_dataset = dataset.map(
        augment_and_make_seed
    )
    return Dataset.zip((dataset, mapped_dataset))


train_datasets: Dataset = train_datasets.apply(apply_augmentation).shuffle(1000).batch(32)

for (image, label) in enumerate(train_datasets.take(5)):
    print(image)
    print(label)



Tensor("args_0:0", shape=(None, None, 3), dtype=uint8)
Tensor("args_1:0", shape=(), dtype=int64)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 0. First element had shape [333,500,3] and element 1 had shape [247,320,3]. [Op:IteratorGetNext] name: 